In [ ]:
!pip install transformers
!pip install torchtext --upgrade

     |████████████████████████████████| 1.5MB 13.4MB/s 
     |████████████████████████████████| 2.9MB 54.4MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=066e17d3e7307fcdf8eaa2daad80dc46824f72997ac0f6433a84134d9b4736b6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 7.0MB 10.5MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [ ]:
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# import BERT 
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data.utils import get_tokenizer

#note we are using the BERT tokenizer for the input
tokenize = lambda x: tokenizer.tokenize(x)
ner_tokenize = lambda x: x.split(' ')

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

TEXT = Field(batch_first = True,
             use_vocab = False,
             tokenize = tokenize,
             preprocessing = tokenizer.convert_tokens_to_ids,
             init_token = init_token_idx,
             eos_token = eos_token_idx,
             pad_token = pad_token_idx,
             unk_token = unk_token_idx)
             #fix_length = 59) #fix_length=max_input_length

# We may use BERT tokenizer or spacy one to split the numbers away
REP = Field(sequential=True, tokenize=ner_tokenize,init_token='<sos>',eos_token='<eos>', lower=True) #Why do we need a fixed length here, really? get_tokenizer("spacy")
#NER = Field(sequential=True, tokenize=ner_tokenize,init_token='<sos>',eos_token='<eos>', lower=True) #may combine with REP, do you need a fix_length at all? Special tokens? 
input_fields = [('text',TEXT),('rep',REP),('ner',REP)]
train_set, validation_set = TabularDataset.splits(path='/content/drive/My Drive/Datasets/',train='samples_ner_ref.csv',validation='validate_ner_ref.csv',format='csv',skip_header=True,fields=input_fields)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warn

In [ ]:
text_lens = []
rep_lens = []
ner_lens = []
for example in train_set.examples:
  text_lens.append(len(example.text))
  rep_lens.append(len(example.rep))
  ner_lens.append(len(example.ner))
print(np.max(text_lens),np.max(rep_lens),np.max(ner_lens))

82 27 82


In [ ]:
text_lens = []
rep_lens = []
for example in validation_set.examples:
  text_lens.append(len(example.text))
  rep_lens.append(len(example.rep))
print(np.max(text_lens),np.max(rep_lens))

81 27


In [ ]:
for example in train_set.examples:
  print(example.text)
  print(example.rep)
  print(example.ner)
  break

[2425, 2033, 1996, 2922, 11868, 2098]
['f_select', 'col_1', 'f_max', 'col_1', 'df_name']
['o', 'o', 'o', 'o', 'col_1', 'col_1']


In [ ]:
#TEXT.build_vocab(train_set,validation_set,min_freq = 1) # tried 10, better with 1 - no need to build vocabulary for BERT 
REP.build_vocab(train_set,validation_set)
#NER.build_vocab(train_set,validation_set)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
REP.vocab.stoi['<eos>']

3

In [ ]:
from torchtext.data import Iterator, BucketIterator

In [ ]:
# this operation does the numerization, <sos>, <eos>, <unk>, <pad> insertion 
train_iter, val_iter = BucketIterator.splits(
 (train_set, validation_set), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(1, 1),device = device) # 16 batch size here for both datasets - LESS IS BETTER (The best one is 1, but super slow )

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
val_iter.sort = False
val_iter.sort_within_batch =  False

In [ ]:
#Previous version 
def get_batch(batch):
    data = batch.text #.transpose(0,1)  #transposing for consistency as BERT preprocessing gives a different dimensionality
    #eos_tensor = torch.tensor([batch.rep,REP.vocab.stoi['<eos>']]*batch.rep.size()[1],device=device)
    #sos_tensor = torch.tensor([batch.rep,REP.vocab.stoi['<sos>']]*batch.rep.size()[1],device=device)
    target_parse = batch.rep
    target_ner = batch.ner
    #target_concat = torch.cat((batch.rep,batch.ner)) #torch.cat((batch.rep,REP.vocab.stoi['<eos>']),dim=1) # + REP.vocab.stoi['<sos>'] + batch.ner
    return data, target_parse, target_ner

In [ ]:
for batch in train_iter:
  data,target_parse,target_ner = get_batch(batch)
  print(data.size(),target_parse.size(),target_ner.size())
  break
#  encoding = bert(data)[0]
#  linear = nn.Linear(768, len(REP.vocab.stoi)).to(device) #encoding.size()[1]
#  print(linear(encoding).size())
  #print(target)
  #pad_id = REP.vocab.stoi['<pad>']
  #print(torch.tensor(target == pad_id,device=device).permute(1,0))
# break

torch.Size([1, 20]) torch.Size([13, 1]) torch.Size([20, 1])


In [ ]:
class TransformerModel(nn.Module):

    def __init__(self,nreptoken, nhead, nhid, nlayers, dropout=0.5): #nnertoken
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer,Transformer
        self.model_type = 'Transformer'
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.bert = bert
        self.src_mask = None
        self.tgt_mask = None
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout)
        #encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        #decoder_layers = TransformerDecoderLayer(ninp, nhead, nhid, dropout)
        ##self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(nreptoken, embedding_dim) #encode to the same dimensionality as bert 
        self.bert_encoder = bert
        self.transformer = Transformer(d_model=embedding_dim, nhead=nhead, num_encoder_layers=nlayers, num_decoder_layers=nlayers, dim_feedforward=nhid, dropout=dropout, activation='relu', custom_encoder=None, custom_decoder=None)
        self.linear = nn.Linear(embedding_dim, nreptoken)
        self.linear_ner = nn.Linear(embedding_dim, nreptoken)
        #self.decoder = TransformerDecoder(decoder_layers, nlayers)
        #self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src,tgt):
        #if self.src_mask is None or self.src_mask.size(0) != len(src):
        #    device = src.device
        #    mask = self._generate_square_subsequent_mask(len(src)).to(device)
        #   self.src_mask = mask
        if self.tgt_mask is None or self.tgt_mask.size(0) != len(tgt):
          device = tgt.device
          mask = self._generate_square_subsequent_mask(len(tgt)).to(device)
          self.tgt_mask = mask

        #get the padding masks
        pad_id = REP.vocab.stoi['<pad>'] 
        src_pad_mask = torch.tensor(src == pad_token_idx)
        tgt_pad_mask = torch.tensor(tgt == pad_id).permute(1,0)

        #print(src.size())
        #with torch.no_grad(): #try to enable for NER purposes 
        src =  self.bert_encoder(src)[0]

        src = src.permute(1,0,2) # you need to permute the bert output because it has flipped batch/seq dimensions, otherwise transformer doesn't comsume it correctly 
        src = self.pos_encoder(src)
        #print(src.size())

        #tgt = tgt.transpose(0,1)
        #with torch.no_grad():
        tgt =  self.encoder(tgt)
        tgt = self.pos_encoder(tgt)

        #output = self.transformer_encoder(src, self.src_mask)
        #print(src.size(),tgt.size())
        
        #,src_key_padding_mask = src_pad_mask, tgt_key_padding_mask = tgt_pad_mask
        output = self.transformer(src = src,tgt = tgt, tgt_mask = self.tgt_mask,src_key_padding_mask = src_pad_mask, tgt_key_padding_mask = tgt_pad_mask) #you need to transpose the bert's output
        output = self.linear(output)
        output_ner = self.linear_ner(src) #should we add a encoder layer here?, probably no
        output = torch.cat((output,output_ner))
        return output

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
nreptokens = len(REP.vocab.stoi) #REP
#nnertokens = len(NER.vocab.stoi) #NER
#emsize = 300 # embedding dimension
nhid = 300 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
model = TransformerModel(nreptokens, nhead, nhid, nlayers, dropout).to(device) #nnertokens,

In [ ]:
#for name, param in model.named_parameters():                
#    if name.startswith('bert'):
#        param.requires_grad = False

In [ ]:
# disabling the training of bert
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 125,589,226 trainable parameters


In [ ]:
criterion = nn.CrossEntropyLoss() # weight=weights, reduction = 'sum', idea - calciulate the weights based on frequency - less frequent get more weight
lr = 0.5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    nreptokens = len(REP.vocab.stoi)
    for i,batch in enumerate(train_iter):
        data, targets_parse,targets_ner = get_batch(batch)
        optimizer.zero_grad()
        output = model(data,targets_parse[:-1, :])
        loss = criterion(output.view(-1, nreptokens), torch.cat((targets_parse[1:, :],targets_ner)).contiguous().view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        #print(output.size())
        total_loss += loss.item()
        log_interval = 50
        if i % log_interval == 0 and i > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_set) // 16, scheduler.get_lr()[0], #
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(REP.vocab.stoi)
    with torch.no_grad():
        for batch in data_source:
            data, targets_parse,targets_ner = get_batch(batch)
            output = eval_model(data,targets_parse[:-1, :])
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, torch.cat((targets_parse[1:, :],targets_ner)).contiguous().view(-1)).item()
    return total_loss / (len(data_source) - 1)

In [ ]:
best_val_loss = float("inf")
epochs = 1 #18 in case of 1 batch reaches 0
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_iter)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |    50/ 2126 batches | lr 0.50 | ms/batch 59.35 | loss  1.39 | ppl     4.03
| epoch   1 |   100/ 2126 batches | lr 0.50 | ms/batch 48.98 | loss  0.60 | ppl     1.82
| epoch   1 |   150/ 2126 batches | lr 0.50 | ms/batch 48.07 | loss  0.42 | ppl     1.52
| epoch   1 |   200/ 2126 batches | lr 0.50 | ms/batch 48.77 | loss  0.43 | ppl     1.53
| epoch   1 |   250/ 2126 batches | lr 0.50 | ms/batch 48.72 | loss  0.39 | ppl     1.48
| epoch   1 |   300/ 2126 batches | lr 0.50 | ms/batch 48.74 | loss  0.33 | ppl     1.38
| epoch   1 |   350/ 2126 batches | lr 0.50 | ms/batch 48.50 | loss  0.35 | ppl     1.43
| epoch   1 |   400/ 2126 batches | lr 0.50 | ms/batch 48.77 | loss  0.30 | ppl     1.34
| epoch   1 |   450/ 2126 batches | lr 0.50 | ms/batch 49.27 | loss  0.27 | ppl     1.31
| epoch   1 |   500/ 2126 batches | lr 0.50 | ms/batch 48.32 | loss  0.22 | ppl     1.25
| epoch   1 |   550/ 2126 batches | lr 0.50 | ms/batch 48.76 | loss  0.24 | ppl     1.27
| epoch   1 |   600/ 

In [ ]:
test = torch.tensor([1])
test = test.repeat(1, 16)
test.size()

torch.Size([1, 16])

In [ ]:
def generate(src,max_length): #src is has the [seq_l] dimensionality 
  #get <pad> id
  pad_id = REP.vocab.stoi['<pad>'] #REP
  eos_id = REP.vocab.stoi['<eos>'] #REP
  sos_id = REP.vocab.stoi['<sos>'] #REP 
  init_tgt = [sos_id]
  eos_counter = 0
  output_ner = []
  output_rep = [] 
  for i in range(max_length):
    tgt_input = init_tgt + [pad_id]*(max_length-len(src))
    tgt_input = torch.tensor(tgt_input).to(device)
    #tgt_input = torch.tensor(list(REP.numericalize([tgt_input]))).to(device)
    output = best_model(src.view(1,len(src)),tgt_input.view(len(tgt_input),1))
    output_ner = output[-len(src):,0,:]
    output_rep = output[:-len(src),0,:]
    _,max_ind = torch.max(output_rep,1)
    rep = [REP.vocab.itos[i] for i in max_ind]
    init_tgt = init_tgt + [max_ind[i]]
    #if max_ind[i] == eos_id:
    #  eos_counter+=1
    #  if eos_counter ==2:
    #    break
    if max_ind[i] == eos_id:
      break
  _,max_ind = torch.max(output_ner,1)
  output_ner_text = [REP.vocab.itos[i] for i in max_ind]
  return ([REP.vocab.itos[i] for i in init_tgt],output_ner_text)

In [ ]:
for batch in val_iter:
  data, targets_parse,targets_ner = get_batch(batch)
  print(data.size(),data[0,:].size())
  #print([TEXT.vocab.itos[i] for i in data[:,0]])
  print("Input:",tokenizer.convert_ids_to_tokens(data[0,:]))
  #generate the representation
  output = generate(data[0,:],40)
  print("Output REP:",output[0])
  print("Output NER:",output[1])
  # generate the right answer
  print("Actual REP:",[REP.vocab.itos[i] for i in batch.rep[:,0]])
  print("Actual NER:",[REP.vocab.itos[i] for i in batch.ner[:,0]])


torch.Size([1, 17]) torch.Size([17])
Input: ['[CLS]', 'name', 'the', 'smallest', 'region', 'where', 'total', 'volume', 'is', 'equal', 'to', '268', '##8', '##31', '.', '64', '[SEP]']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Output REP: ['<sos>', 'f_select', 'col_1', 'f_min', 'col_1', 'f_filter', 'df_name', 'f_eq', 'col_2', 'var_2', '<eos>', 'f_eq', 'col_2', 'var_2', '<eos>']
Output NER: ['<sos>', 'o', 'o', 'o', 'col_1', 'o', 'col_2', 'col_2', 'o', 'o', 'o', 'var_2', 'var_2', 'var_2', 'var_2', 'var_2', '<eos>']
Actual REP: ['<sos>', 'f_select', 'col_1', 'f_min', 'col_1', 'f_filter', 'df_name', 'f_eq', 'col_2', 'var_2', '<eos>']
Actual NER: ['<sos>', 'o', 'o', 'o', 'col_1', 'o', 'col_2', 'col_2', 'o', 'o', 'o', 'var_2', 'var_2', 'var_2', 'var_2', 'var_2', '<eos>']
torch.Size([1, 56]) torch.Size([56])
Input: ['[CLS]', 'name', 'the', 'largest', 'month', 'which', 'have', '8th', 'st', 'at', 'bran', '##nan', 'st', 'end', '_', 'station', '_', 'name', 'or', '1992', '.', '0', 'member', '_', 'birth', '_', 'year', 'or', 'member', '_', 'birth', '_', 'year', 'being', '1989', '.', '0', 'or', 'having', 'powell', 'st', 'bart', 'station', '(', 'market', 'st', 'at', '4th', 'st', ')', 'start', '_', 'station', '_', 'name', '[

KeyboardInterrupt: ignored

In [ ]:
#test random input

In [ ]:
test_input ='Name the most expensive animal_name with 124567 legs or 8 tentacles or fast speed'
#test_input ='Name the largest fat_legs with 1 predator'
fix_length = 59
#test_input ='Which priority has the most entries?'
#test_input ='What are unique priority'
src_input = tokenize(test_input)
src_input = ['[CLS]'] + src_input + ['[SEP]']
print('BERT tokenization:',src_input)
src_input = tokenizer.convert_tokens_to_ids(src_input)
#print('BERT decoded:',tokenizer.decode(src_input))
src_input = torch.tensor(src_input).to(device)
output = generate(src_input,fix_length)
print("REP:",output[0])
print("NER:",output[1])
ner_output = np.array(output[1])
entity_values = list(ner_output[ner_output!= 'o'][1:-1]) #cut sos and eos 
entity_indexes = list(np.where(ner_output != 'o')[0][1:-1])
key_tokens = {}
for i,index in enumerate(list(entity_indexes)):
  token_list = key_tokens.get(entity_values[i],[])
  token_list.append(index)
  key_tokens[entity_values[i]] = token_list
for key in key_tokens:
  tokens = [src_input.cpu().numpy()[i] for i in key_tokens[key]]
  token_str = tokenizer.decode(tokens).replace(' _ ','_') #join words if underscore is between them
  key_tokens[key] = token_str 
print(key_tokens)

BERT tokenization: ['[CLS]', 'name', 'the', 'most', 'expensive', 'animal', '_', 'name', 'with', '124', '##56', '##7', 'legs', 'or', '8', 'tentacles', 'or', 'fast', 'speed', '[SEP]']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


REP: ['<sos>', 'f_select', 'col_1', 'f_max', 'price', 'f_filter', 'df_name', 'f_or', 'f_eq', 'col_3', 'var_3', 'f_or', 'f_eq', 'col_4', 'var_4', 'f_eq', 'col_2', 'var_2', '<eos>']
NER: ['<sos>', 'o', 'o', 'o', 'o', 'col_1', 'col_1', 'col_1', 'o', 'var_2', 'var_2', 'var_2', 'col_2', 'o', 'var_3', 'col_3', 'o', 'var_4', 'col_4', '<eos>']
{'col_1': 'animal_name', 'var_2': '124567', 'col_2': 'legs', 'var_3': '8', 'col_3': 'tentacles', 'var_4': 'fast', 'col_4': 'speed'}


In [ ]:
torch.save(best_model.state_dict(), '/content/drive/My Drive/Models/model_14062020.state')

In [ ]:
import pickle
pickle.dump(REP.vocab, open( "/content/drive/My Drive/Models/vocab.pickle", "wb" ))